# Algoritmia
## Práctica Obligatoria
 2 - Divide y Vencerás
### Curso 2021 - 2022
---
 

#### Autores:
---
Resuelve la práctica de acuerdo al enunciado. Se proporciona un esqueleto de los métodos a utilizar que es necesario completar. El código proporcionado no puede modificarse (sí que pueden añadirse nuevos métodos)

**Importante**: 
* Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.6/library/index.html), **salvo las funciones de ordenación.**
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [158]:
# imports
import math

In [159]:
# Versión ligeramente modificada de la clase Video de la Práctica 1

# Se ha incluido una propiedad 'fee', correspondiente a los costes 
# semanales de licenciamiento del contenido
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size, fee):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        fee : number
            Coste de alquiler
        """
        self.name = name
        self.size = size
        self.fee = fee
        self.users = dict()
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
        return hash(str(self))
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return "(" + str(self.name) + ", " + str(self.size) + ", " + str(self.fee) + ")"
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return "(" + str(self.name) + ", " + str(self.size) + ", " + str(self.fee) + ")"
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        
        self.users[country] = users
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """        
        return self.users[country]

    
    def __gt__(self, other):
        return (self.size > other.size)
    
    def __lt__(self, other):
        return (self.size < other.size)
    

In [160]:
def key_sort(seq, key_function=lambda x:x, reverse=False):
    """Ordena ascendentemente una secuencia utilizando una función clave.

    Parameters
    ----------
    seq : sequence
        Secuencia a ordenar, por ejemplo una lista
    key_function : int
        Función que devuelve para los elementos de la secuencia un valor
        clave para la ordenación
    reverse: boolean
        Si es True Invierte el orden de la ordenación.
    
    Notes
    -----
    No tiene retorno. Ordena en el sitio la secuencia de entrada.
    """      
    mergesort(seq, key_function, reverse)

    # if reverse:
    #     seq.reverse()

def mergesort(seq, key_function, reverse):
    if len(seq)<=1:
        return
    mitad = len(seq) // 2
    izq = seq[:mitad]
    der = seq[mitad:]

    mergesort(izq, key_function, reverse)
    mergesort(der, key_function, reverse)
    merge(izq, der, seq, key_function, reverse)

def comparar(a, b, key_function):
    if key_function(a) == key_function(b):
        return a < b
    return key_function(a) < key_function(b)
        

def merge(izq, der, seq, key_function, reverse):
    i = j = k = 0

    while i < len(izq) and j < len(der):
        if comparar(izq[i], der[j], key_function) ^ reverse:
            seq[k] = izq[i]
            i += 1
        else:
            seq[k] = der[j]
            j += 1
        k += 1

    while i < len(izq):
        seq[k] = izq[i]
        i += 1
        k += 1

    while j < len(der):
        seq[k] = der[j]
        j += 1
        k += 1

In [161]:
def max_emission_profit(video, forecast, ingresos_por_usuario, method="bruteforce"):
    """Calcula el máximo beneficio obtenible emitiendo un vídeo según 
       la previsión de espectadores disponible.

    Parameters
    ----------
    video : Video
        Vídeo a emitir.
    forecast : iterable
        Previsión de espectadores.
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    method: String
        Algoritmo a emplear 'bruteforce' o 'divideandconquer'.
    
    Returns
    -------
    float
        Máximo beneficio obtenible.
        
    Notes
    -----
    El máximo beneficio se calcula considerando la mejor combinación 
    posible de momento de inicio y finalización dentro del periodo de 
    forecast, el coste de licenciamiento del vídeo (fee) y los beneficios 
    obtenidos por cada espectador. Si el vídeo no puede emitirse de forma
    rentable, el método devuelve 0.
    'video' debe disponer de una propiedad 'fee' que establece los costes
    de licenciamiento en periodos equivalentes a las previsiones de 'forecast'.
    """      
    if method == "bruteforce":
        return bruteforce(video, forecast, ingresos_por_usuario)
    elif method == "divideandconquer":
        nueva_lista = [i * ingresos_por_usuario - video.fee for i in forecast]
        return max_emission_profit_divideandconquer(nueva_lista, 0, len(nueva_lista) - 1)
    else:
        raise ValueError("Unknown method: " + method)


def bruteforce(video, forecast, ingresos_por_usuario):
    max = -999999

    for l in range(0,len(forecast)):
        val = mejorCombinacion(video, l, forecast, ingresos_por_usuario)
        if val > max:
            max = val
    if max < 0:
        return 0
    return max

def mejorCombinacion(video, semanas, forecast, ingresos_por_usuario):
    max = -99999
    for i in range(0, len(forecast) - semanas):
        total = 0
        for prevision in forecast[i:i+semanas + 1]:
            total -= video.fee
            total += prevision * ingresos_por_usuario
        if total > max:
            max = total
    return max

def max_emission_profit_divideandconquer(seq, izq, der):
    if izq == der:
        return seq[izq]

    medio = (izq + der) // 2
    suma_izq = max_emission_profit_divideandconquer(seq, izq, medio)
    suma_der = max_emission_profit_divideandconquer(seq, medio + 1, der)
    suma_cruz = suma_cruzada(seq, izq, medio, der)
    sum = max(suma_izq, suma_der, suma_cruz)
    if sum < 0:
        return 0
    return sum

def suma_cruzada(seq, izq, medio, der):
    suma = 0
    suma_izq = -9999999999
    suma_der = -9999999999
    for i in range(medio, izq - 1, -1):
        suma += seq[i]
        if suma > suma_izq:
            suma_izq = suma
    suma = 0
    for i in range(medio + 1, der + 1):
        suma += seq[i]
        if suma > suma_der:
            suma_der = suma
    return suma_izq + suma_der
        

In [162]:
def optimize_content_grid(videos, forecasts, ingresos_por_usuario, k=None):
    """Obtiene los vídeos rentables para su emisión según la previsión
    de espectadores y los ingresos por usuario.

    Parameters
    ----------
    videos : list of Video
        Vídeos a analizar.
    forecasts : list of list of int
        Secuencia de previsiones para los vídeos
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    k: int
        Obtiene los k elementos más rentables.
    
    Returns
    -------
    iterable<Video>
        k videos rentables
        
    Notes
    -----
    La rentabilidad de los vídeos se considera tomando el periodo
    óptimo de emisión para cada uno. El método sólo devuelve resultados
    con rentabilidad mayor que 0.
    """      

    listaVideos = list(tuple())
    # key_sort(listaViedos, lambda x: max_emission_profit(x, forecasts[videos.index(x)], ingresos_por_usuario, method="divideandconquer"), True)

    for i in range(len(videos)):
        profit = max_emission_profit(videos[i], forecasts[i], ingresos_por_usuario, method="divideandconquer")
        if profit <= 0:
            continue
        listaVideos = insert(listaVideos, (videos[i], profit))

    if k != None:
        return [i[0] for i in listaVideos[:k]]
    return [i[0] for i in listaVideos]

def insert(list, n):
    index = len(list)
    for i in range(len(list)):
        if list[i][1] < n[1]:
            index = i
            break

    if index == len(list):
        list = list[:index] + [n]
    else:
        list = list[:index] + [n] + list[index:]
    return list

## Pruebas de ejemplo

Superar estas pruebas no garantiza la corrección de la implementación efectuada.

In [163]:
import unittest,random

# Vídeos de ejemplo para tests
videos_test = [Video("Vikings: Valhalla", 150, 1500), #Título, tamaño, fee
               Video("New Amsterdam", 56, 980),
               Video("Juego de Tronos",152, 2500),
               Video("La casa de papel", 76, 800),
               Video("Breaking Bad", 160, 1230),
               Video("Pasión de gavilanes", 125, 350),
               Video("The Sinner", 89, 1900),
               Video("El Cid", 35, 410),
               Video("Raised by wolves",80, 1900),
               Video("Euphoria",90, 2000),
               Video("Peacemaker",40, 1000)]

# Audiencias previstas para los anteriores
forecasts_test = [[ 500, 1000, 2000,  150, 2000, 1500,  200, 1200,  700],
                  [1000,  800,  500,  250, 4000,  500,  100,  200,  300],
                  [1200, 1000,  900,  900, 1220, 1100,  900, 1000, 1000],
                  [ 900,  700,  500,  200, 1600,  200,  300,  250,  100],
                  [ 600,  900, 1240,  250, 2100, 1000,  350, 1000,  800],
                  [ 190, 1060,  500,  200,  250,  700,  200,  400,  400],
                  [ 150, 1100, 2500,  600,  800, 3000,  200, 1000,  900],
                  [ 160,  100,  200,  200,  150,  200,   50,  200,  190],
                  [ 900,  850,  960,  950, 1100,  100,  350,  900,  800],
                  [ 600,  900, 1200,  300, 1900, 1000,  650,  850,  800],
                  [ 510,  500,  500,  500,  500,  500,  500,  500,  510]]

class TestKeySort(unittest.TestCase):
    
    def test_basic_sort(self):
        v = list(range(100))
        random.seed = 1234
        random.shuffle(v)
        key_sort(v)
        self.assertEqual(v, list(range(100))) 

    def test_key_sort(self):
        v = list(range(100))
        key_sort(v, key_function=lambda x:-x)
        self.assertEqual(v, list(reversed(range(100))))    
    
    def test_video_sort(self):
        v = videos_test.copy()
        key_sort(v) # ordena por el criterio de comparación definido en Video (tamaño)
        self.assertEqual(v,list(sorted(videos_test)))
        
        key_sort(v, key_function=lambda x:x.fee, reverse=True)
        self.assertEqual(v,list(sorted(videos_test,key=lambda x:x.fee, reverse = True)))

class TestOptimizer(unittest.TestCase):
    
    def test_max_profit(self):   
        for m in ["divideandconquer"]:
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [2]*10, 1, method=m), 10)

    def test_max_profit_dataset(self):      
        results = {1: [500, 3020, 0, 800, 870, 960, 1100, 0, 0, 0, 0],
                   2: [5800, 8220, 0, 3800, 5440, 4650, 6500, 0, 320, 1800, 40]}
        for m in ["bruteforce", "divideandconquer"]:
            for r in results:
                for v,f,g in zip(videos_test, forecasts_test, results[r]):
                    self.assertEqual(g, max_emission_profit(v, f, r,  method=m))
    
    def test_optimize_content_grid(self):
        test_result_k2 = ['New Amsterdam', 'The Sinner', 'Vikings: Valhalla', 'Breaking Bad', 'Pasión de gavilanes', 
                          'La casa de papel', 'Euphoria', 'Raised by wolves', 'Peacemaker']
            
        a = [v.name for v in optimize_content_grid(videos_test, forecasts_test, 2)]
        self.assertEqual(test_result_k2,
                         a)


In [164]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.030s

OK
